In [1]:
import pandas as pd
import json
import os

In [2]:
data_path = "../local_tests_data/raw_articles_list"

all_articles = {}

for filename in os.listdir(data_path):

    if not filename.endswith(".json"):
        continue


    with open(os.path.join(data_path, filename), "r") as f:
        data = json.load(f)

    for article in data:
        article_id = article.get("article_id")
        all_articles[article_id] = article

In [3]:
output_filename = 'batch_file_relevance_check_0_OUTPUT.jsonl'

In [ ]:
with open(output_filename, "r") as file:
    lines_raw = file.readlines()
    for line in lines_raw:
        output_dict = json.loads(line)
        model = output_dict.get("response").get("body").get("model")
        line_id = output_dict.get("custom_id")
        content_json = output_dict.get("response").get("body").get("choices")[0].get("message").get("content")
        content = json.loads(content_json)
        # relevance check version references check logic (e.g. prompt)
        task_name, article_id = line_id.split("*")
        all_articles[article_id]["relevance"] = content.get("relevance", "N/A")
        all_articles[article_id]["relevance_check_model"] = model
        all_articles[article_id]["relevance_check_version"] = task_name

In [5]:
pd_article_relevance = pd.DataFrame.from_dict(all_articles, orient='index').reset_index()

In [6]:
pd_article_relevance.head()

,index,title,url,keywords,author,article_id,source,relevance,relevance_check_model,relevance_check_version
0,the_next_web-20250528111956213260,How to thrive with AI agents — tips from an HP...,https://thenextweb.com/news/tips-on-ai-agents,"[AI, strategy, HP]",None,the_next_web-20250528111956213260,the_next_web,2,gpt-4o-2024-11-20,relevance-check-v0
1,the_next_web-20250528111956213288,Meta AI chief: ‘Inferiority complex’ is stunti...,https://thenextweb.com/news/european-tech-stru...,"[Meta, AI, European tech]",None,the_next_web-20250528111956213288,the_next_web,2,gpt-4o-2024-11-20,relevance-check-v0
2,the_next_web-20250528111956213298,US shoplifting ‘epidemic’ sparks demand for Fr...,https://thenextweb.com/news/us-shoplifting-epi...,"[shoplifting, AI cameras, France]",None,the_next_web-20250528111956213298,the_next_web,2,gpt-4o-2024-11-20,relevance-check-v0
3,the_next_web-20250528111956213306,TNW Backstage dives into the mind-bending worl...,https://thenextweb.com/news/tnw-backstage-brai...,"[brain-computer interfaces, technology, innova...",None,the_next_web-20250528111956213306,the_next_web,2,gpt-4o-2024-11-20,relevance-check-v0
4,the_next_web-20250528111956213312,Startup uses ancient bacteria to turn Texas CO...,https://thenextweb.com/news/startup-uses-ancie...,"[sustainability, CO2, green chemicals]",None,the_next_web-20250528111956213312,the_next_web,2,gpt-4o-2024-11-20,relevance-check-v0


In [7]:
pd_article_relevance['relevance'].value_counts()

relevance
2    43
1    18
0     1
Name: count, dtype: int64

In [8]:
with open("../local_tests_data/relevant_articles/relevant_articles.json", "w") as f:
    json.dump(pd_article_relevance[pd_article_relevance.relevance == 2].to_dict(orient='records'), f, indent=4, ensure_ascii=False)